## Построить граф миграций

In [ ]:
from network_construction import NetworkConstruction
import pandas as pd
import pickle

In [ ]:
# Указать путь к файлам

path = ''

In [ ]:
# Чтение необходимых файлов

vacancies = pd.read_csv(path + "vacancy_unique_cities.csv")
responses = pd.read_csv(path + "responses.csv", sep=";")
cv_cities = pd.read_csv(path + "curricula_vitae_cities.csv")
cv_years = pd.read_csv(path + "curricula_vitae_years.csv")
kladr_2021 = pd.read_csv(path + "kladr.csv")
organizations = pd.read_csv(path + "organizations.csv", delimiter=";")

with open(path + "orgs_from_dadata_all.pkl", "rb") as f:
    org_cities = pickle.load(f)
org_cities = pd.DataFrame(org_cities)
org_cities = pd.json_normalize(org_cities.iloc[:, 0])

specialists = pd.read_csv(path + 'specialists_table.csv')
ontology = pd.read_csv(path + 'ontology.csv')

In [ ]:
# data preprocessing

responses = responses.merge(specialists[["id_cv", "label"]], how="left").merge(
    ontology.rename(columns={"Специальность": "label", "Отрасль": "industry"})[
        ["label", "industry"]
    ]
    .drop_duplicates()
    .groupby("label")["industry"]
    .apply(lambda x: ",".join(x))
    .reset_index()[["label", "industry"]],
    how="left",
)

responses.loc[:, "industry"].fillna("Unknown", inplace=True)

#### Отфильтровать по году и индустрии исходные таблицы

In [ ]:
G, responses2 = NetworkConstruction.run_pipeline(
    responses=responses,
    vacancies=vacancies,
    kladr_2021=kladr_2021,
    organizations=organizations,
    dadata_organisations_info=org_cities,
    cv_years=cv_years,
    cv_cities=cv_cities,
    years=list(range(2015, 2022)),
    industries=["Судостроение"],
    return_responses=True,
)

#### Отфильтровать по профессии обработанную таблицу responses

In [ ]:
G2, responses3 = NetworkConstruction.run_pipeline(
    pre_calculated_responses=responses2,
    group=['Монтажник'],
    return_responses=True
)

#### Просто построить граф по обрабоданной таблице responses

In [ ]:
G3 = NetworkConstruction.run_pipeline(
    pre_calculated_responses=responses3
)